In [ ]:
%pip install torch torchvision matplotlib pandas 
%pip install transformers


In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import timm
from timm.scheduler import CosineLRScheduler
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch.nn as nn
import time


from torch.utils.data import DataLoader

# The device is currently optimized for MPS (Metal Performance Shaders) on macOS.
# Feel free to change it to yours locally
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
train_set = datasets.Food101("../data", download=True)
test_set = datasets.Food101("../data", split="test", download=True)

classes = train_set.classes
print(classes)

img_count = len(train_set._image_files)
# Print 6 random images
for _ in range(6):
  index = np.random.choice(img_count, replace=False)
  plt.imshow(train_set[index][0])
  plt.title(f"Label: {classes[train_set[index][1]]}")
  plt.show()

In [ ]:
# Resize the images to 224x224
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
])

test_transform = transforms.Compose([
  transforms.Resize((256, 256)),
  transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)
])

train_set.transform = train_transform
test_set.transform = test_transform

for _ in range(6):
  index = np.random.choice(img_count, replace=False)
  plt.imshow(train_set[index][0].permute(1, 2, 0).numpy())
  plt.title(f"Label: {classes[train_set[index][1]]}")
  plt.show()

In [ ]:
# Prepare the model
# Load from Timm instead of huggingface (Had an error)
model = timm.create_model(
    'deit_small_patch16_224.fb_in1k', 
    pretrained=True,num_classes=len(classes),
    drop_rate=0.3,
    drop_path_rate=0.1,)
model = model.to(device)

# I want to test the output of the pretrained on a random image (PROTOTYPE REASONS ONLY)
index = np.random.choice(img_count, replace=False)
img, label = train_set[index]
img = img.to(device).unsqueeze(0) 
output = model(img)
print(f"Output shape: {output.shape}")
print(f"Predicted class index: {output.argmax(dim=1).item()}")
print(f"Predicted class label: {classes[output.argmax(dim=1).item()]}")
print(f"True class label: {classes[label]}")

In [ ]:
# Split dataset 
print(f"Train size: {len(train_set)}, Test size: {len(test_set)}")

In [ ]:
# Data Loader
batch_size = 48

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

In [ ]:
# Cross entropy loss and Adam Optimizer
loss = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

epochs = 25
warmup_epochs = 3

scheduler = CosineLRScheduler(
    optimizer,
    t_initial=epochs,
    lr_min=1e-6,
    warmup_lr_init=1e-5,
    warmup_t=warmup_epochs,
    cycle_limit=1,
    t_in_epochs=True,
)


In [ ]:
print(f"Training for {epochs} epochs...")
for epoch in range(epochs):
    start_time = time.time()
    print(
        f"Epoch {epoch+1} started at {time.strftime('%H:%M:%S', time.localtime(start_time))}"
    )
    # Training loop above
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss_value = loss(outputs, labels)
        loss_value.backward()
        optimizer.step()

        running_loss += loss_value.item()
    progress = (epoch + 1) / epochs * 100

    scheduler.step(epoch + 1)
    end_time = time.time()
    print(f"Epoch {epoch+1} ended at {time.strftime('%H:%M:%S', time.localtime(end_time))}")
    print(f"Time taken for epoch {epoch+1}: {end_time - start_time:.2f} seconds")
    print(f"Progress: {progress:.2f}%")
    print(f"Epoch [{epoch+1}/{epochs}], LR: {scheduler._get_lr(epoch)[0]:.6f}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# eval
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")
# Accuracy on 1 epoch: 59%
# Accuracy on 6 epoch: 69.81%